In [1]:
import os
import torch
import numpy as np
from torch import nn
from torchvision import utils,models,transforms,datasets
from torch.utils.data import DataLoader
import tensorflow as tf
import random
from keras.datasets import mnist,cifar10
import PIL
from PIL import Image
import shutil
from google.colab import drive
import torch.nn.functional as F

In [2]:
(train_x,train_y),(test_x,test_y) = cifar10.load_data()
train_x = torch.from_numpy(train_x)
train_y = torch.from_numpy(train_y)
test_x = torch.from_numpy(test_x)
test_y = torch.from_numpy(test_y)

train_x = train_x.permute([0,3,1,2]) / 255
train_y = train_y.squeeze(1)
test_x = test_x.permute([0,3,1,2]) / 255
test_y = test_y.squeeze(1)

170498071/170498071 [==============================] - 4s 0us/step


In [3]:
train_x.shape

torch.Size([50000, 3, 32, 32])

In [4]:
train_y.shape

torch.Size([50000])

In [5]:
test_x.shape

torch.Size([10000, 3, 32, 32])

In [6]:
test_y.shape

torch.Size([10000])

In [ ]:
def tensor_to_image(tensor):
    transform = transforms.ToPILImage()
    return transform(tensor)

In [ ]:
class SimulatedClient():
    def __init__(self,client_id):
        self.id = client_id

    def load_parameters(self):
        file_path = f"./clients/client_{self.id}/parameters.txt"
        with open(file_path, 'r') as file:
            for line in file:
                key, value = line.strip().split(': ')
                if key == 'battery_power':
                    self.battery = int(value)
                elif key == 'service_type':
                    self.comm_type = int(value)
                elif key == 'service_strength':
                    self.comm_strength = float(value)
                elif key == 'computing_power':
                    self.comp_capability = float(value)


    def create_dataloader(self):
        classes_name = ['Airplane','Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse','Ship','Truck']
        images = torch.empty(0,3,32,32)
        labels = torch.empty(0)
        count = 0
        data_dir = f"/content/clients/client_{self.id}/image/"
        for cls in range(len(classes_name)):
            dir = os.path.join(data_dir,f"{classes_name[cls]}")
            filelist = os.listdir(dir)
            image_files = []
            for file2 in filelist:
                if file2.lower().endswith('.png'):
                    image_files.append(file2)
            for i in image_files:
                image_path = os.path.join(dir,i)
                try:
                    with Image.open(image_path) as image:
                        transform = transforms.ToTensor()
                        image_tensor = transform(image)
                        image_tensor -= 0.5
                        image_tensor /= 0.5
                        images = torch.cat((images,image_tensor.unsqueeze(0)))
                        label = torch.tensor(cls)
                        label = label.unsqueeze(0)
                        labels = torch.cat((labels,label))
                except Exception as e:
                    print()
        dl = DataLoader((list(zip(images,labels))), batch_size = 16, shuffle = True, num_workers = 0)
        return dl

    #advk
    def update_data(self,images,labels):
        classes_name = ['Airplane','Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse','Ship','Truck']
        try:
            # Your existing code for updating data using images and labels

            # Extract the folder path from the file_path
            folder_path = f"/content/clients/client_{self.id}/image/"
            # folder_path = "/".join(file_path.split("/")[:-1])

            # Remove the folder and its contents
            shutil.rmtree(folder_path)
        except Exception as e:
            print(f"Error: {e}")

        client_directory=f"/content/clients/client_{self.id}/"
        create_image_folder(client_directory)
        # Add the new data to the 'images' directories
        for k in range(len(images)):
            image = tensor_to_image(images[k])
            label = labels[k].data
            digit_folder = os.path.join(folder_path, f'{classes_name[label]}')
            image.save(os.path.join(digit_folder, f"image{k}.png"))


#advk end
    def train(self,model,optimizer,criterion):
        model.train()
        dataloader = self.create_dataloader()
        if len(dataloader) == 0:
            return model

        running_loss = 0.0
        running_corrects = 0
        size = 0
        for data,target in dataloader:
            data = data
            target = target.long()
            optimizer.zero_grad()

            output = model(data)
            loss = criterion(output,target)
            _,preds = torch.max(output,1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * data.size(0)
            # print(preds,target.data)
            running_corrects += torch.sum(preds == target.data)
            size += data.size(0)

        epoch_loss = running_loss / size
        epoch_acc = running_corrects.double() / size

        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        return model



In [ ]:
def create_image_folder(client_directory):
    # Create 'image' folder within the client directory
    image_folder = os.path.join(client_directory, 'image')
    if not os.path.exists(image_folder):
        os.makedirs(image_folder)


    # Create folders numbered 'digit1' to 'digit10' within the 'image' folder
    add_digit_folders(image_folder)


def add_digit_folders(image_folder):
    # Add 10 'digit' folders within the 'image' folder
    classes_name = ['Airplane','Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse','Ship','Truck']
    for i in range(0, 10):
        digit_folder = os.path.join(image_folder, f'{classes_name[i]}')
        if not os.path.exists(digit_folder):
            os.makedirs(digit_folder)

In [ ]:
# Number of clients
num_clients = 100  # You can change this to the desired number of clients

#creating clients
clients = [SimulatedClient(i) for i in range(num_clients)]

In [ ]:

# Parent directory to store client data folders
parent_directory = "clients"

# Create the parent directory if it doesn't exist
os.makedirs(parent_directory, exist_ok=True)

classes_name = ['Airplane','Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse','Ship','Truck']

for client_id in range(num_clients):
    # Generate random data for each client
    battery_power = random.randint(1, 100)
    service_type = random.randint(0, 1)
    service_strength = random.uniform(0,1)
    computing_power = random.uniform(0,1)

    # Create a dictionary with client data
    client_data = {
        "client_id": client_id,
        "battery_power": battery_power,
        "service_type": service_type,
        "service_strength": service_strength,
        "computing_power": computing_power
    }


    # Create a folder for each client within the parent directory
    client_directory = os.path.join(parent_directory, f"client_{client_id}")
    os.makedirs(client_directory, exist_ok=True)

    # Create a text file within each client folder and write data to it
    file_path = os.path.join(client_directory, f"parameters.txt")
    with open(file_path, 'w') as file:
        for key, value in client_data.items():
            file.write(f"{key}: {value}\n")

    create_image_folder(client_directory)

    images_x = train_x[client_id*500:(client_id+1)*500]
    images_y = train_y[client_id*500:(client_id+1)*500]

    for k in range(500):
        trial = images_x[k]
        image = tensor_to_image(trial)
        label = images_y[k].data
        digit_folder = os.path.join(client_directory, 'image', f'{classes_name[label]}')
        image.save(os.path.join(digit_folder, f"image{k}.png"))

    # reading all data from the file
    clients[client_id].load_parameters()



In [ ]:
num_clients_select=20
def select_clients():
  selected_clients = random.sample(clients, num_clients_select)

  return selected_clients

In [ ]:
train_x -= 0.5
train_x /= 0.5
test_x -= 0.5
test_x /= 0.5
train_dataloader = DataLoader(list(zip(train_x,train_y)),shuffle = True, batch_size = 16)
val_dataloader = DataLoader(list(zip(test_x,test_y)),shuffle = True, batch_size = 16)

In [ ]:
def federated_averaging_sequential(global_model, client_model,weight):

    # Initialize an accumulator to store the aggregated model parameters
    aggregated_params = {var_name: param.data*weight for var_name,param in global_model.named_parameters()}

    # Aggregate model parameters from each client
    for var_name, var in client_model.named_parameters():
        aggregated_params[var_name] += var.data

    # Compute the average of the model parameters
    averaged_params = {var_name: param / (weight+1) for var_name,param in aggregated_params.items()}

    updated_global_model = global_model
    for var_name, param in updated_global_model.named_parameters():
        param.data = averaged_params[var_name]

    return updated_global_model


In [ ]:
class CNNModel(nn.Module):

  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 10, kernel_size = 5, stride = 1)
    self.conv2 = nn.Conv2d(in_channels = 10, out_channels = 10, kernel_size = 5, stride = 1)
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.lin1 = nn.Linear(in_features= 10*5*5,out_features = 100)
    self.lin2 = nn.Linear(100,10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = torch.flatten(x,1)
    x = F.relu(self.lin1(x))
    x = self.lin2(x)
    return x

In [ ]:
num_epochs = 50
client_selection = []
criterion_federated = nn.CrossEntropyLoss()

model_classifier_federated = CNNModel()
optimizer_federated = torch.optim.SGD(model_classifier_federated.parameters(),lr = 0.01,momentum = 0.9)

# model_classifier_federated = model_classifier_federated.cuda()
weight = 1

images = train_x[25000:]
labels = train_y[25000:]

for epoch in range(num_epochs):
    # selecting clients to train model on
    client_selection = select_clients()
    for i in clients:
        model = i.train(model_classifier_federated,optimizer_federated,criterion_federated)


        model_classifier_federated = federated_averaging_sequential(model_classifier_federated,model,weight)

        # if weight != 1000:
            # i.update_data(images[(weight-1)*25:weight*25],labels[(weight-1)*25:weight*25])
        weight += 1


    running_loss = 0.0
    running_corrects = 0
    model_classifier_federated.eval()
    for data,target in val_dataloader:
        data = data
        target = target
        optimizer_federated.zero_grad()

        output = model_classifier_federated(data)
        loss = criterion_federated(output,target)
        _,pred = torch.max(output,1)

        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(pred == target.data)

    epoch_loss = running_loss / test_x.shape[0]
    epoch_acc = running_corrects.double() / test_x.shape[0]



    print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')




In [ ]:
epochs = 25

model_classifier_central = CNNModel()
optimizer_central = torch.optim.SGD(model_classifier_central.parameters(),lr = 0.01, momentum = 0.9) # keeping hyperparameters same
criterion_central = nn.CrossEntropyLoss()

# model_classifier_central = model_classifier_central

for epoch in range(epochs):
    running_loss = 0.0
    running_corrects = 0
    model_classifier_central.train()

    for data,target in train_dataloader:
        data = data
        target = target
        optimizer_central.zero_grad()

        output = model_classifier_central(data)
        loss = criterion_central(output,target)
        _,preds = torch.max(output,1)

        loss.backward()
        optimizer_central.step()

        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target.data)

    epoch_loss = running_loss / train_x.shape[0]
    epoch_acc = running_corrects.double() / train_x.shape[0]

    print(epoch)
    print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    running_loss = 0.0
    running_corrects = 0
    model_classifier_central.eval()
    for data,target in val_dataloader:

        data = data
        target = target
        optimizer_central.zero_grad()

        output = model_classifier_central(data)
        loss = criterion_central(output,target)
        _,preds = torch.max(output,1)

        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target.data)

    epoch_loss = running_loss / test_x.shape[0]
    epoch_acc = running_corrects.double() / test_x.shape[0]


    print(f'Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')



0
Train Loss: 1.6145 Acc: 0.4090
Val Loss: 1.4393 Acc: 0.4900
1
Train Loss: 1.3770 Acc: 0.5113
Val Loss: 1.4070 Acc: 0.5085
